<a href="https://colab.research.google.com/github/mateusz-wozny/IMO/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [5]:
kroa100 = pd.read_csv('kroA100.txt', sep=' ', header=None)

matrx = np.array(kroa100.iloc[:, 1])
matry = np.array(kroa100.iloc[:, 2])
N = matrx.shape[0]
a = np.ones((N, 1))
kron = np.kron(a, matrx)
X = (kron - kron.T) ** 2
krony = np.kron(a, matry)
Y = (krony - krony.T) ** 2
dist_mat = np.uint(np.sqrt(X + Y).round(0))

In [6]:
dist_mat

array([[   0, 1693, 2252, ...,  783, 1468, 2643],
       [1693,    0, 1708, ..., 1911, 3156, 1831],
       [2252, 1708,    0, ..., 1786, 3315,  454],
       ...,
       [ 783, 1911, 1786, ...,    0, 1535, 2228],
       [1468, 3156, 3315, ..., 1535,    0, 3760],
       [2643, 1831,  454, ..., 2228, 3760,    0]], dtype=uint64)

In [ ]:
def index(dist, lista, nieodw):
    df = dist[:, lista][nieodw, :]
    ind, minim = np.unravel_index(df.argmin(), df.shape)
    return nieodw[ind], lista[minim]
def nearest_neighbor(dist_mat, start):
    end = np.argmax(dist_mat[start, :].flatten())
    nieodw = list(range(dist_mat.shape[0]))
    nieodw.remove(start)
    nieodw.remove(end)
    list1 = [start]
    list2 = [end]
    while len(nieodw) != 0:
        ind, minim = index(dist_mat, list1, nieodw)
        list1.insert(list1.index(minim) + 1, ind)
        nieodw.remove(ind)
        ind, minim = index(dist_mat, list2, nieodw)
        list2.insert(list2.index(minim) + 1, ind)
        nieodw.remove(ind)
    list1.append(start)
    list2.append(end)
    koszt = sum([dist_mat[list1[i], list1[i+1]] for i in range(len(list1)-1)])+sum(
        [dist_mat[list2[i], list2[i+1]] for i in range(len(list2)-1)])

    return int(koszt)

In [ ]:
calk_koszt = []
K = 30

for _ in range(K):
    start = random.randint(0,N-1)
    # print(dist_mat[0:3, :])
    koszt = nearest_neighbor(dist_mat, start)
    # koszt = random_heuristic(dist_mat)
    calk_koszt.append(koszt)
print(round(sum(calk_koszt) / K), min(calk_koszt), max(calk_koszt))

32940 28782 35151


In [ ]:
def random_heuristic(dist_mat):
    N = dist_mat.shape[0]
    a = random.sample(range(N), N)
    list1 = a[:N//2]
    list1.append(list1[0])
    list2 = a[N//2:]
    list2.append(list2[0])
    koszt = sum([dist_mat[list1[i], list1[i + 1]] for i in range(len(list1) - 1)]) + sum(
        [dist_mat[list2[i], list2[i + 1]] for i in range(len(list2) - 1)])
    plt.plot(kroa100.loc[list1].iloc[:, 1], kroa100.loc[list1].iloc[:, 2])
    plt.plot(kroa100.loc[list2].iloc[:, 1], kroa100.loc[list2].iloc[:, 2])
    plt.scatter(kroa100.iloc[:, 1], kroa100.iloc[:, 2])
    plt.title(f'Koszt={koszt}')
    plt.show()
    return int(koszt)